## Test solver

Environment

In [1]:
import sys
sys.path.append("./../../../../")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import os
import numpy as np

from ronek import postproc as pp
from ronek.systems import TASystem

In [3]:
import matplotlib as mpl
# Use custom style
# See: https://matplotlib.org/1.5.3/users/style_sheets.html
stylefile = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle"
mpl.style.use(stylefile)

Define inputs

In [4]:
# System temperature for isothermal bath simulation [K]
T = 1e4
# Parameterized initial conditions:
mu = [
  3e2,   # Initial equilibrium temperature [K]
  5e-2,  # Initial molar concentration of atoms
  1e3    # Initial pressure [Pa]
]
# Time limits for the simulation [s]
tlim = [1e-9, 1e-4]
# Paths for database and solution storage
path_to_dtb = "./../../database/"
path_to_saving = "./solution/"

In [5]:
os.makedirs(path_to_saving, exist_ok=True)

Initialize master equation model

In [6]:
system_qct = TASystem(
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  rates_coeff=path_to_dtb + "/kinetics_qct.hdf5",
  use_einsum=False,
  use_factorial=True,
  use_arrhenius=False
)
system_qct.update_fom_ops(T)

In [7]:
# system_mdl = TASystem(
#   species={
#     k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
#   },
#   rates_coeff=..., # Path to the mean predicted rates
#   use_einsum=False,
#   use_factorial=True,
#   use_arrhenius=False
# )
# system_mdl.update_fom_ops(T)

Solve isothermal heat bath simulation

In [8]:
t = system_qct.get_tgrid(*np.sort(tlim), 500)
n0 = system_qct.mix.get_init_sol(*mu, mu_type="mole")

> QCT

In [9]:
ni_qct = system_qct.solve_fom(t, n0)[1]

> Model

In [10]:
# ni_mdl = system_mdl.solve_fom(t, n0)[1]
ni_mdl = ni_qct

Postprocessing
> Moments of the distribution

In [11]:
pp.plot_mom_evolution(
  path=path_to_saving,
  t=t,
  n_m={"QCT": ni_qct, "Model": ni_mdl},
  molecule=system_qct.mix.species["molecule"],
  max_mom=2,
  molecule_label="\\text{O$_2$}",
  ylim_err=None,
  err_scale="log",
  hline=1.0,
  tlim=tlim
)